In [ ]:
#
# _____ ______  _______      _  _  _ _______ _______ _______  _____  __   _      _     _
#   |   |_____] |  |  |      |  |  | |_____|    |    |______ |     | | \  |       \___/ 
# __|__ |_____] |  |  |      |__|__| |     |    |    ______| |_____| |  \_|      _/   \_                                                                                       
#

In [ ]:
!pip install "tracing_auto_instrumentation[ibm]" --upgrade

In [ ]:
import logging
import os

from enum import Enum

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

from lastmile_eval.rag.debugger.api import LastMileTracer
from lastmile_eval.rag.debugger.tracing.sdk import get_lastmile_tracer

In [ ]:
import dotenv
dotenv.load_dotenv()

In [ ]:
logger = logging.getLogger(__name__)

In [ ]:
class Mode(Enum):
    GENERATE = "GENERATE"
    GENERATE_TEXT = "GENERATE_TEXT"

def init_watson_model() -> Model:
    # To display example params enter
    GenParams().get_example_values()
    generate_params = {GenParams.MAX_NEW_TOKENS: 25}

    watson_model = Model(
        model_id=ModelTypes.GRANITE_13B_CHAT_V2,
        params=generate_params,
        credentials=dict(
            api_key=os.getenv("WATSONX_API_KEY"),
            url="https://us-south.ml.cloud.ibm.com",
        ),
        space_id=os.getenv("WATSONX_SPACE_ID"),
        verify=None,
        validate=True,
    )

    return watson_model

In [ ]:
from tracing_auto_instrumentation.ibm import wrap_watson

TRACE_NAME = "elementary-my-dear-watson"

tracer: LastMileTracer = get_lastmile_tracer(
    TRACE_NAME
)
watson_model: Model = init_watson_model()
wrapper = wrap_watson(watson_model, tracer)

In [ ]:
def run_generation(mode: Mode, prompt: str) -> None:
    logger.info("IBM WatsonX Generation Script Starting...")

    # n.b. required for multiple api keys, make sure `.env` is in your local path
    dotenv.load_dotenv()
    
    response: str = ""
    if Mode.GENERATE == mode:
        logger.info(f"running with mode: {mode}")
        response = wrapper.generate(prompt)
    elif Mode.GENERATE_TEXT == mode:
        logger.info(f"running with mode: {mode}")
        response = wrapper.generate_text(prompt)
    else:
        logger.error(f"unsupported mode: {mode}")
    logger.info(f"response: {response}")

In [ ]:
response = run_generation(
    mode=Mode.GENERATE,
    prompt="Can you please tell me an amusing anecdote about Thomas Edison?",
)

response